In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import anodeclstmgru.constants as const
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact

In [ ]:
store = pd.HDFStore(const.HDF_STORE_PATH)
df_phys_norm_v1 = store['df_phys_norm_v1']
store.close()

In [ ]:
df_phys_norm_v1 = df_phys_norm_v1.set_index('Timestamp', drop=True)
df_phys_norm_v1 = df_phys_norm_v1.drop(' Timestamp', axis=1)
sensors = df_phys_norm_v1.columns
day_list_normal = list(range(22, 29))+['all']
df_phys_norm_v1.head()

In [ ]:
def get_single_time_series(sensor, day=28, sample_freq='s'):
    df_plot = df_phys_norm_v1.copy()
    if not day == 'all':
        df_plot = df_plot[(df_plot.index > pd.Timestamp(f'2015-12-{day}')) &
                         (df_plot.index < pd.Timestamp(f'2015-12-{day+1}'))]
    df_plot = df_plot.resample(sample_freq).mean()
    title = f'{sensor} over time (averaged over {sample_freq})'
    fig = px.line(df_plot, x=df_plot.index, y=sensor, title=title)
    fig.show()
interact(get_single_time_series, sensor=sensors, day=day_list_normal,
         sample_freq=['1s', '5s', '10s', '60s'])

In [ ]:
def get_4_signal_plot(sensor0='FIT101', sensor1='LIT101',
                      sensor2='LIT301', sensor3='LIT401', day=28, sample_freq='s'):
    df_plot = df_phys_norm_v1.copy()
    if not day == 'all':
        df_plot = df_plot[(df_plot.index > pd.Timestamp(f'2015-12-{day}')) &
                         (df_plot.index < pd.Timestamp(f'2015-12-{day+1}'))]
    df_plot = df_plot.resample(sample_freq).mean()
    
    sensor_selection_list=[sensor0, sensor1, sensor2, sensor3]
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    for i, s in enumerate(sensor_selection_list):
        fig.add_trace(
            go.Scatter(x=df_plot.index,
                       y=df_plot[s], name=s),
            row=i+1, col=1,
        )
    title = f'{sensor0}, {sensor1}, {sensor2} and {sensor3}'\
            f' over time (averaged over {sample_freq})'

    fig.update_layout(height=600, width=800, title_text=title)
    fig.show()

interact(get_4_signal_plot, sensor0=sensors, sensor1=sensors,
         sensor2=sensors, sensor3=sensors, day=day_list_normal,
         sample_freq=['1s', '5s', '10s', '60s'])

In [ ]:
!which python